In [10]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [2]:
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4-1106-preview")

In [3]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

--2024-07-24 11:25:40--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260185 (1.2M) [application/octet-stream]
Saving to: ‘data/10q/uber_10q_march_2022.pdf’

data/10q/uber_10q_m 100%[===================>]   1.20M  6.40MB/s    in 0.2s    

2024-07-24 11:25:40 (6.40 MB/s) - ‘data/10q/uber_10q_march_2022.pdf’ saved [1260185/1260185]

--2024-07-24 11:25:41--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [4]:
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

In [5]:
import os


def get_tool(name, full_name, documents=None):
    if not os.path.exists(f"./data/{name}"):
        # build vector index
        vector_index = VectorStoreIndex.from_documents(documents)
        vector_index.storage_context.persist(persist_dir=f"./data/{name}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{name}"),
        )
    query_engine = vector_index.as_query_engine(similarity_top_k=3, llm=llm)
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name,
            description=(
                "Provides information about Uber quarterly financials ending"
                f" {full_name}"
            ),
        ),
    )
    return query_engine_tool

In [8]:

march_tool = get_tool("march_2022", "March 2022", documents=march_2022)
june_tool = get_tool("june_2022", "June 2022", documents=june_2022)
sept_tool = get_tool("sept_2022", "September 2022", documents=sept_2022)

In [14]:
query_engine_tools = [march_tool]

In [1]:
from llama_index.core.agent import AgentRunner, ReActAgent
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
from llama_index.agent.openai import OpenAIAgentWorker
import os
from llama_index.llms.openai import OpenAI
# openai_step_engine = OpenAIAgentWorker.from_tools(
#     query_engine_tools, llm=llm, verbose=True
# )
# agent = AgentRunner(openai_step_engine)
# # alternative
# agent = OpenAIAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

#agent_llm = OpenAI(model="gpt-3.5-turbo")
agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    query_engine_tools, llm=agent_llm, verbose=True, max_iterations=20
)

NameError: name 'query_engine_tools' is not defined

In [20]:
response = agent.chat("Analyze the changes in R&D expenditures and revenue")

Thought: The current language of the user is English. I need to use a tool to help me answer the question about Uber's R&D expenditures and revenue changes.
Action: march_2022
Action Input: {'input': 'R&D expenditures and revenue changes'}


Retrying llama_index.llms.openai.base.OpenAI._chat in 0.2624219115644425 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.3876672051702936 seconds as it raised APIConnectionError: Connection error..


Observation: Error: Connection error.
Thought: Since there is a connection error with the tool, I cannot use it to retrieve the information. I will have to inform the user that I cannot provide the requested analysis with the provided tools.
Answer: I am currently unable to access the specific financial data due to a connection error with the tool. To analyze changes in R&D expenditures and revenue, I would typically need access to Uber's financial reports over multiple quarters or years. Since I cannot retrieve this information at the moment, I recommend checking Uber's investor relations website or financial news sources for the latest financial reports and conducting the analysis based on the data provided there.


In [21]:
print(str(response))

I am currently unable to access the specific financial data due to a connection error with the tool. To analyze changes in R&D expenditures and revenue, I would typically need access to Uber's financial reports over multiple quarters or years. Since I cannot retrieve this information at the moment, I recommend checking Uber's investor relations website or financial news sources for the latest financial reports and conducting the analysis based on the data provided there.


In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
import os
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.openai import OpenAI

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_path = '/home/sanka/NLP_models/model/mistral-7b-instruct-v0.1.Q4_0.gguf', verbose=False,
    temperature=0.9
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    
)

os.environ['OPENAI_API_KEY'] =""
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)
#add_tool = FunctionTool(fn=add)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

def devide(a: float, b: float) -> float:
    """devide two numbers and returns the answer"""
    return a / b

add_tool = FunctionTool.from_defaults(fn=add)
devide_tool = FunctionTool.from_defaults(fn=devide)
llm = OpenAI(model="gpt-4-1106-preview")
#llm = Ollama(model="/home/sanka/NLP_models/model/mistral-7b-instruct-v0.1.Q4_0.gguf", request_timeout=120.0)
agent = ReActAgent.from_tools([multiply_tool, add_tool, devide_tool], llm=llm, verbose=True)

response = agent.chat("What is 20+(2*4)/3? Calculate step by step.")

print(response)

Thought: The user has asked for a step-by-step calculation of the expression 20+(2*4)/3. I will break down the calculation into its components and use the appropriate tools to solve each part.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: I have the result of the multiplication part of the expression, which is 8. Now I need to divide this result by 3.
Action: devide
Action Input: {'a': 8, 'b': 3}
Observation: 2.6666666666666665
Thought: I have the result of the division part of the expression, which is approximately 2.67. Now I need to add this result to 20.
Action: add
Action Input: {'a': 20, 'b': 2.6666666666666665}
Observation: 22.666666666666668
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of the expression 20+(2*4)/3 is approximately 22.67. Here's the step-by-step calculation:

1. Multiply 2 by 4 to get 8.
2. Divide 8 by 3 to get approximately 2.67.
3. Add 20 to the result from step 2 to ge